In [ ]:
%pip install -r requirements.txt

# Imports

In [1]:
import os
import json

from typing import List
from dotenv import load_dotenv

import requests

In [2]:
from pydantic import BaseModel, Field

In [3]:
from langchain.chains import SimpleSequentialChain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap

In [4]:
from langchain_openai import AzureChatOpenAI

In [5]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langfuse import Langfuse

# Load Config

In [8]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
langfuse_public = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_secret = os.getenv("LANGFUSE_SECRET_KEY")

In [9]:
# Set the environment variables
os.environ['AZURE_OPENAI_API_KEY'] = azure_api_key
os.environ['AZURE_OPENAI_ENDPOINT'] = azure_endpoint

# LLM Configuration

In [10]:
# Azure OpenAI - GPT-4o or GPT-4o-mini
llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name="gpt-4o",
    temperature=0.3,
    api_version="2024-12-01-preview",
)

# Output Schema

In [11]:
# Define the output schema using Pydantic
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float = Field(ge=0.0, le=1.0)

# Parser
parser = PydanticOutputParser(pydantic_object=SentimentOutput)

# Initialize Langfuse

In [12]:
# Initialize Langfuse
langfuse = Langfuse(public_key=langfuse_public, secret_key=langfuse_secret)

In [13]:
# Verify connection, do not use in production as this is a synchronous call
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


# Get Relevant Data

In [14]:
# Fetch symbol from company name
def get_stock_ticker(company_name):
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=url, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [15]:
# Fetch Yahoo Finance news
def fetch_news(ticker: str) -> str:
    tool = YahooFinanceNewsTool()
    print("Yahoo Finance Ticker : ", ticker.upper())
    return tool.invoke(ticker)

# Setup Chain

In [16]:
# Step 4: Sentiment analysis prompt
sentiment_prompt = PromptTemplate(
    input_variables=["company", "ticker", "news"],
    template="""
Analyze the sentiment of the news for the given company.

Company: {company}
Ticker: {ticker}
News Summaries:
{news}

Return the result using this JSON format:
{format_instructions}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [17]:
# Final sentiment chain
sentiment_chain = sentiment_prompt | llm | parser

# Main Pipeline Function

In [18]:
def run_sentiment_pipeline(company_name: str):
    span = langfuse.start_span(name="sentiment_analysis_pipeline", input={"company": company_name})

    # Extract ticker
    ticker = get_stock_ticker(company_name)
    span.update(name="stock_code_extraction", input=company_name, output=ticker)

    # Fetch news
    news = fetch_news(ticker)
    span.update(name="news_fetching", input=ticker, output=news)

    # Run sentiment analysis
    input_data = {
        "company": company_name,
        "ticker": ticker,
        "news": news
    }
    result = sentiment_chain.invoke(input_data)
    span.update(name="sentiment_parsing", input=input_data, output=result)
    span.end()
    return result

# Test Case

In [19]:
company = input("Enter company name: ")
print("Running chain for company : ", company.upper())
result = run_sentiment_pipeline(company)
print(json.dumps(result.dict(), indent=2))

Running chain for company :  APPLE
Yahoo Finance Ticker :  AAPL


ImportError: Could not import yfinance python package. Please install it with `pip install yfinance`.

In [20]:
# Flush all pending observations
langfuse.flush()
langfuse.shutdown()